<h1>Pseudocode for a minimax algorithm (Adapt this for our game)</h1>

<h1> Nim Rules (for reference) </h1>

<h2> To Dos </h2>

1) We already have a great minimax pseudocode--it's really not that complicated. We just need to make a tree for Nim and adapt the minimax algorithm as needed

    1b) Test the Tree!

<h1> Nim Rules & Tree Setup </h1>

In [1]:
import copy

In [2]:
class Stack:
    def __init__(self, size):
        self.size = size
    
    def removeStone(self, amount):
        if amount <= self.size and amount != 0:
            self.size -= amount
            return True #Indicates a valid turn
        
        return False #Indicates an invalid turn

In [3]:
nim = [Stack(3), Stack(3), Stack(3)]

In [4]:
for stack in nim: print(stack.size)

3
3
3


In [5]:
class Node: #For ease, TRUE is the MAX player and FALSE is the MIN player
    def __init__(self, state, children=None, parent=None, player=None): #Might need parent...but at the same time, do we even care about the parent? Would we use it?
        self.player = player
        self.state = state
        self.parent = parent
        self.children = children
        if children != None:
            for child in self.children:
                child.parent = self
                child.player = !self.player

In [6]:
def children_populate(nim):
    children = []
    stack_index = 0
    for each in nim:
        for i in range(1, each.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, children_populate(temp_nim)))
        stack_index += 1
    return children

#This no longer works per changing the children to Nodes instead of Stacks
test_children = children_populate(nim)
for stack in nim: print(stack.size)
for child in test_children:
    for stack in child: print(stack.size, end = " ")
    print("\n")

In [7]:
class NimTree:
    def __init__(self, nim):
        self.nim = nim
        self.origin = Node(nim, player=False) #Theoretically, because children are recursively created in the children_populate function...this should work. Theoertically.
        self.origin.children = children_populate(self.origin.state)

In [ ]:
nim = [Stack(3), Stack(3), Stack(3)] #After putting in all the player stuff, compilation takes a LONG time. Not sure if that's cuz of bool size or..?
tree = NimTree(nim)

#Okay Marcus, I'll explain some of what I've done thus far.
#Nim is a game w/ a collection of an arbitrary number of stacks & an arbitrary number of stones per stack
#Players take turns removing stones (they can remove as many as they want / can (no negative stones) or as a few as they want / can but HAVE to remove 1)
#So what I've done is thus:
#The Stack class acts as a stack
#nim is a starting state of the game, represented by an array of stacks.
#Node is simply a node containing a current state of nim (an array of stacks) that also links to its children
#I didn't include parent because I don't think we need it and the way I populated the tree was a little recursive and wonky, and it didn't play well w/ parent. We can fix this pretty easily though.
#NimTree is a class containing, you guessed it, a tree of Nim Nodes. 
#Currently, I think it's working right. However, I haven't had time to test it. Do you think you can test and troubleshoot the tree? Should be able to do this with a for loop
#It should look like slide 14 of chapter 7 notes.
#You can use the following lines of code to print nicely

In [ ]:
for stack in tree.origin.state: print(stack.size, end = " ")
print("\n") #this prints the nim state of the origin node of the tree

#Outdated since children are Nodes.
for child in tree.origin.children:
    for stack in child: print(stack.size, end = " ")
    print("\n")

In [ ]:
for child in tree.origin.children: #This prints the nim state of all of the children nodes of a node (in this case, origin)
    for stack in child.state: print(stack.size, end = " ")
    print("\n")

In [ ]:
def print_all_nodes(node):
    global count
    children = node.children
    for child in children:
        print_all_nodes(child)
    
    print("Node: ")
    for stack in node.state: 
        print(stack.size, end = " ")
    print("\n")
    
    print("Player: "):
        if (node.player):
            print("MAX")
        else:
            print("MIN")

    if(node.parent != None):
        print("Parent: ")
        for stack in node.parent.state: print(stack.size, end = " ")
        print("\n")

    if(node.children != []):
        print("Children: ")
        for child in children:
            for stack in child.state: 
                print(stack.size, end = " ")
            print("\n")
    print("------------------")
    count += 1

In [ ]:
print_all_nodes(tree.origin) #We're getting some repeated nodes...I think this is how it should be though?
print(count)

TO DO FOR MARCUS
Psuedocode:
for each node in tree:
    verify proper children (if any) are being generated per slide 14 of chapter 7
    also test larger nim games (both stack # and stack size)
    
    Maybe start looking at incorporating the minimax algorith? Shouldn't be too hard w/ a tree in place
    
You'll do great :) Just get done what you can and we can keep working on it together.

Also, as a tip:
click Cell->run all when you start & periodicailly if you update code.
Shift + Enter runs one block
Cell->Cell Type->Markdown is good for writing notes like this.

<h1> MiniMax Tree </h1>

<p1> Okay so the idea here is that win for MAX is denoted a 1 and the win for MIN is denoted a 0. Once we have that setup, we need to define some sort of heuristic for game AI to choose the odds of it getting a 1 more. We could probably just use a simple ratio here. I think we can calculate up the scores pretty easily...hold tight. </p1>

In [ ]:
# https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-1-introduction/
# A simple Python3 program to find 
# maximum score that 
# maximizing player can get 
import math 
  
def minimax (curDepth, nodeIndex, 
             maxTurn, scores,  
             targetDepth): 
  
    # base case : targetDepth reached 
    if (curDepth == targetDepth):  
        return scores[nodeIndex] 
      
    if (maxTurn): 
        return max(minimax(curDepth + 1, nodeIndex * 2,  
                    False, scores, targetDepth),  
                   minimax(curDepth + 1, nodeIndex * 2 + 1,  
                    False, scores, targetDepth)) 
      
    else: 
        return min(minimax(curDepth + 1, nodeIndex * 2,  
                     True, scores, targetDepth),  
                   minimax(curDepth + 1, nodeIndex * 2 + 1,  
                     True, scores, targetDepth)) 
      
# Driver code 
scores = [3, 5, 2, 9, 12, 5, 23, 23] 
  
treeDepth = math.log(len(scores), 2) 
  
print("The optimal value is : ", end = "") 
print(minimax(0, 0, True, scores, treeDepth)) 
  
# This code is contributed 
# by rootshadow 

<h1> User Input & Test Playing </h1>